In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


**Learning outcomes**

To understand:
* Mapping iterators (recap)
* Accumulating iterators
* Iterators versus loops (do and while)
* Combining Iterators

# Introduction
An iterator (previously known as adverbs) modifies a function's behaviour - specifically it changes the function's application so that it is applied **iteratively**. There are very few explicit loops in kdb+/q (you may have noticed theres no such thing as a "for" loop!) and this is because we use iterators instead! 

The primary methods of Iteration in q:
1. [Atomic](https://code.kx.com/q/basics/atomic/) Functions
2. [Mapping](https://code.kx.com/q/basics/iteration/#maps) iterators - each and its variants
3. [Accumulating](https://code.kx.com/q/basics/iteration/#accumulators) iterators - scan / over

1 and 2 have been covered in our KX Fundamentals course but we will do a quick recap before diving into Accumulators.

# Atomic Functions
Many q functions natively iterate recursively through list or dictionary arguments down to items of some depth. 

These atomic functions **do not** require iterators, applying them is redundant.

In [3]:
neg 1 2 3 // neg natively performs iteration over a list

-1 -2 -3


In [4]:
neg each 1 2 3 // each is redundent here

-1 -2 -3


Note for this to work the arguments of an atomic function must be conformable i.e. of same length.

In [ ]:
1 2 3 + 100 1000  // get a 'length error

In [5]:
1 2 3 + 100 1000 10000 // this works as lists of same length

101 1002 10003


A useful way to know if you require an iterator or not is to ask yourself is my input shape the same as my output shape?

In [6]:
neg(1 2;77 88 99) // input shape = output shape

-1 -2
-77 -88 -99


For functions that do an aggregation (list into an atom) we know this can not be atomic as the input shape is different to output shape.

In [10]:
count (1 2; 10 100 1000) // count operates at top level - input shape = list and output shape = atom

2


In [12]:
count each (1 2; 10 100 1000) // to count at the nested level we will use the map iterator each

2 3


##### Exercise 

Calculate the total price of sym using ```dict:`aapl`msft`kx!(10.5 12.4 13.0 15.1;9.3 10.2 14.5 15.6;3.2 4.6 6.1 8.6)```

In [ ]:
dict:`aapl`msft`kx!(10.5 12.4 13.0 15.1;9.3 10.2 14.5 15.6;3.2 4.6 6.1 8.6)
sum each dict

//write your code here
dict:`aapl`msft`kx!(10.5 12.4 13.0 15.1;9.3 10.2 14.5 15.6;3.2 4.6 6.1 8.6)
sum each dict

# Mapping iterators
These are the most straight forward iterators in kdb+/q - mapping iterators modify a functions application to iterate across every item in a list and are potentially the closest we come in kdb+ to "for" loops. 

| Maps      |  |
|:----------- | :----------- |
| Each | each       |
| Each - Both | ' |
| Each - Left   | \:        |
| Each - Right   | /:        |
| Each - Prior   | ':        |

This iteration is explicit in our naming of these operators, all of which are known as some variant of "each": 

* each - (`each`) - used to apply a function to each item in a list - similar to "for x in list; do function(x)" in other languages 

In [3]:
show L:(1 2 3;10 20;30 40 50;60)
count L
count each L //count is a function that only takes one argument

4
3 2 3 1
1 2 3
10 20
30 40 50
60


What happens if we apply ```each``` to a function with more that one argument?

In [ ]:
3 in each L  //returns error as each only works on monadic functions

* each-both - (`'`) - the dyadic iterator modifies a functions operation to use the items from two lists of the same length in a pairwise fashion (or using a list and an atom).

In [28]:
L where 3 in'(1 2 3;10 20;30 40 50;60) //This dyadic iterator can be applied to a dyadic function.

1 2 3


In [11]:
1 2 3 4 5 in '(3 4 6 4 9)

00010b


* each left - (`\:` note leaning left at top) - apply the provided function to each item in the left-hand-side list
* each right - (`/:` note leaning right at top) - apply the provided function to each item in the right-hand-side list

In [27]:
3 4 in/: (3;10 20;30 4 50;60) //checking for both 3 and 4 in L
//3 4 ,\: L  

10b
00b
01b
00b


* each prior - (`':` or `prior`) - apply a function using subsequent pairs within a list

In [30]:
-':[200 300 100 500 400 200 100 -200 400 -100]
prior[-;200 300 100 500 400 200 100 -200 400 -100]
deltas 200 300 100 500 400 200 100 -200 400 -100

200 100 -200 400 -100 -200 -100 -300 600 -500
200 100 -200 400 -100 -200 -100 -300 600 -500
200 100 -200 400 -100 -200 -100 -300 600 -500


##### Exercise 

Using mapping iterators,  create a file path for each of the files contained in the ipynb_checkpoints folder. 

The file path should look like: 
````:.ipynb_checkpoints/Iterators exercises.ipynb. ``` 

In [ ]:
cp:hsym[first key `:.] //checkpoint directory
cp,/: key cp           //joining directory with each file
` sv'cp,/: key cp      //making our full file paths for each file within that directory 

In [35]:
//write your code here
first key `:.
show cp:hsym[first key `:.] //checkpoint directory
key cp
cp,/: key cp           //joining directory with each file
` sv'cp,/: key cp      //making our full file paths for each file within that directory 

.ipynb_checkpoints
`s#,`Iterators-checkpoint.ipynb
:.ipynb_checkpoints Iterators-checkpoint.ipynb
,`:.ipynb_checkpoints/Iterators-checkpoint.ipynb
`:.ipynb_checkpoints


# Accumulating iterators

[Accumulation](https://code.kx.com/q/ref/accumulators/) iterators execute repeatedly over the values returned from calling the modified function. The function is first applied to the entire(first) argument of the derived function next to the result of the evaluation then to **that** result and so on.


There are two accumulator iterations in kdb+/q, both of which operate in the exact same fashion, excepting their return value. These functions are: 

| Accumulators      |  |
|:----------- | :----------- |
| [scan](https://code.kx.com/q/ref/accumulators/#unary-application) | /       |
| [over](https://code.kx.com/q/ref/accumulators/#binary-application) | \ | 

Using the image below let's look at how scan works:

 <img src="../images/scanIteration.png" width="500" height="500">

## Scan

The iterator [scan](https://code.kx.com/q/ref/accumulators/#unary-application), denoted by `\` (backslash) or the reserved word `scan`, has many behaviours, one of which is to  modify a dyadic or monadic function.

In mathematical notation, the output of a scan function is as follows:
\begin{align}
y_0=x_0 
,y_n=f(y_{n-1};x_n)
\end{align}

Where $x_n$ is the $n^{th}$ element of the input list, $y_n$ is the $n^{th}$ element of the returned output list, and $f$ is our function. 

Examples are perhaps the most illustrative:

In [38]:
(+/)1 4 7 10                       //solution is (1) (1)+4 (1+4)+7 (1+4+7)+10 

22


If we wanted to use the keyword `scan` instead we use the below syntax.

In [37]:
(+) scan 1 2 3 4 5                  //note that this is the same as sums

1 3 6 10 15


Again, this consecutive summing is such a popular operation that there is a keyword dedicated to it - [`sums`](https://code.kx.com/q/ref/sum/). 

In [39]:
sums 1 2 3 4 5 
sums                                //we can see the definition of sums

1 3 6 10 15
+\


If we wanted to append one element of a list to the next to a list in each iteration, we can do so using scan: 

In [40]:
(,\)(1 +til 5)

,1
1 2
1 2 3
1 2 3 4
1 2 3 4 5


Does scan work for all data types? Let's look at we can manipulate strings with scan: 

In [41]:
ssr\["hello word."; ("h";".";"rd"); ("H";"!";"rld")]

"Hello word."
"Hello word!"
"Hello world!"


##### Worked Example 

This iterator isn't solely reserved for use with inbuilt function, it can also be applied to our functions. 

Lets work through an example of compounding interest on money in a savings account:

**Given**: savings account balance, interest rate

**Find**: a function that will tell us our yearly return 

In [42]:
yearlyReturn:{[savingsBalance;interest] rate: 1+0.01*interest;  //multiplying is a "cheaper" operation than division
                        savingsBalance*rate}  
yearlyReturn[10000;1.2]   //account balance at the end of one year with the at 1.2% savings interest rate 

10120f


Let's start with an estimation of the next few years interest rates, and for comparison purposes lets have a look at the S&P annual change over the last five years: 

In [43]:
interestRates: 1 1 1.25 2 1.6 1.5            //2016, 2017, 2018, 2019, 2020 YTD per Ally bank - https://www.consumerismcommentary.com/rates/
last5yearsSnP: 9.54 19.42 -6.24 28.88 -5.77  //2016, 2017, 2018, 2019, 2020 YTD - https://www.macrotrends.net/2526/sp-500-historical-annual-returns

We can calculate the compounding interest on this by providing a starting account balance and then supplying the subsequent rates:

In [44]:
(yearlyReturn\) 10000f,interestRates   //compounding at saving rate over last 5 years
(yearlyReturn\) 10000f,last5yearsSnP   //compounding at S&P return over last 5 years

10000 10100 10201 10328.51 10535.08 10703.64 10864.2
10000 10954 13081.27 12265 15807.13 14895.06


In [45]:
//providing a "seed" or "initial" value is common and can be done using the below syntax either: 
(yearlyReturn\)[10000f;last5yearsSnP]

10954 13081.27 12265 15807.13 14895.06


Compounding is important because we see at the end we have quite a big difference!

##### Exercise 

Using the table below for a US tour, find the route in where you start in New York and finish in Austin.

In [47]:
(`Austin<>)route\`NewYork  //New York to Austin

`NewYork`Washington`Miami`Austin


In [46]:
show USTour:([]StartCity:`Boston`NewYork`Washington`Miami`Austin`Chicago;EndCity:`NewYork`Washington`Miami`Austin`Chicago`Boston;wp:0 1 1 1 1 0)
show route:USTour[`StartCity]!USTour[`EndCity]

StartCity  EndCity    wp
------------------------
Boston     NewYork    0 
NewYork    Washington 1 
Washington Miami      1 
Miami      Austin     1 
Austin     Chicago    1 
Chicago    Boston     0 
Boston    | NewYork
NewYork   | Washington
Washington| Miami
Miami     | Austin
Austin    | Chicago
Chicago   | Boston


In [48]:
//write your code here

(`Austin<>)route\`NewYork

`NewYork`Washington`Miami`Austin


## Over

The adverb [`over`](https://code.kx.com/q/ref/accumulators/#binary-application), denoted by `/` (forward slash) or the reserved word `over`, can also be used to modify a dyadic or monadic function. It operates in the same way as scan, but `over` only returns the **last item** that `scan` produces. 

In [49]:
(+/)1 2 3 4 5
(+) over 1 2 3 4 5

15
15


In [53]:
til 5

0 1 2 3 4


In [56]:
sum 1 2 3 4 5
(,) scan (1 +til 5)

15
,1
1 2
1 2 3
1 2 3 4
1 2 3 4 5


In [55]:
(yearlyReturn/)[10000f;interestRates]
(yearlyReturn/)[10000f;last5yearsSnP]

10864.2
14895.06


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> If puzzled by the result of using <code>over</code>, try replacing it with <code>scan</code> and examining the intermediate results - they are usually illuminating!</i></p>

##### Exercise

Using over, create a dyadic function that will take ``1 2 3 4 5 6 7 8 9`` as an input and will return `45`. The function should print out the following values after each iteration.  

```
    0 1
    1 2
    3 3
    6 4
    10 5
    15 6
    21 7
    28 8
    36 9
```

In [ ]:
//write your code here
addList:{[x;y]0N!(x;y);x+y}
0 addList/ 1 2 3 4 5 6 7 8 9

In [57]:
//write your code here
addList:{[x;y]0N!(x;y);x+y}
0 addList/ 1 2 3 4 5 6 7 8 9

45
0 1
1 2
3 3
6 4
10 5
15 6
21 7
28 8
36 9


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> While <code>scan</code> and <code>over</code> perform the same computation, in general, <code>over</code> requires less memory, because it does not store intermediate results.

##### Quiz Time!
Try Execrises 1.1 and 1.2 to have a quick `scan` `over` your iterator knowledge!

## Overloaded iterators

The accumulating iterators can used to create three different applications in which we will discuss further in this section: 

| Application | Functional notation      |
|:----------- | :----------- |
| [Converge](https://code.kx.com/q/ref/accumulators/#converge)       | v1\\[x]| 
| [Do](https://code.kx.com/q/ref/accumulators/#do) |v1\\[i;x] | 
| [While](https://code.kx.com/q/ref/accumulators/#while) |v1\\[t;x] | 

### Converge 

Using scan, we can let the function iterator over until the result is either the previous result or the input to the function:  

In [60]:
{x*x}\[0.1]            //one input to the function
(rotate[1]\)"hello"    //also works on strings

0.1 0.01 0.0001 1e-08 1e-16 1e-32 1e-64 1e-128 1e-256 0
"hello"
"elloh"
"llohe"
"lohel"
"ohell"


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Remember not all sequences converge- we could create an infinite loop quite quickly! To prohibit an infinite loop from locking a kdb+ session,we can use <code>\T </code> command. </i></p>

### Do and While

As mentioned in the next section, [do](https://code.kx.com/q/ref/do/) and [while](https://code.kx.com/q/ref/while/) are keywords in kdb+/q however we can achieve the same results and better performance using `scan` and `over`. 

Using the left argument, we can define how many times we want to iterate:  

In [61]:
dbl:2*         //user defined function
3 dbl\2 7      //iterating over 3 times 

2  7 
4  14
8  28
16 56


Given a criteria on the left-hand side of the function, we can define that we want to iterate until it doesn't fulfill that criteria:

In [62]:
{last[x]<20} dbl\2 7

2 7 
4 14
8 28


##### Fibonacci sequence

Let's work towards generating the first 10 numbers of the Fibonacci sequence 

> The Fibonacci sequence is defined such that each number is the sum of the two preceding ones, starting from 0 and 1. e.g. the first four numbers in the Fibonacci sequence is: 0 1 1 2

In [ ]:
//recap concepts 
2#til 10
-2#til 10
1 2,10 20 30

Let's look at how we can create a function to calculate the Fibonacci sequence using the keywords above:

In [ ]:
{-2#x}0 1             // retrieving the last two items in the list
{sum -2#x}0 1         // summing them
{x,sum -2#x}0 1       // joining the list to the summation of the last two items 

In [67]:
{x,sum -2#x}\[10;0 1]  // using scan in "do" mode
{x,sum -2#x}/[10;0 1]  // using over in "do" mode 

0 1
0 1 1
0 1 1 2
0 1 1 2 3
0 1 1 2 3 5
0 1 1 2 3 5 8
0 1 1 2 3 5 8 13
0 1 1 2 3 5 8 13 21
0 1 1 2 3 5 8 13 21 34
0 1 1 2 3 5 8 13 21 34 55
0 1 1 2 3 5 8 13 21 34 55 89
0 1 1 2 3 5 8 13 21 34 55 89


##### Exercise

Create a sequence of prime numbers from 0 to 20. 

In [63]:
{$[min y mod x;x,y;x]}/[2;3+til 20]

2 3 5 7 11 13 17 19


In [ ]:
{$[min y mod x;x,y;x]}/[2;3+til 20]

##### Quiz Time!
`do` Exercise 1.3 `while` the overloaded iterators are still fresh in your memory!

# Iterators versus Loops (do and while)

The control words [`do`](https://code.kx.com/q/ref/do/) and [`while`](https://code.kx.com/q/ref/while/) allow q programmers to write explicit loops. However loops/iteration can almost always be defined using iterators, yielding code that is shorter, faster and less prone to error.

Suppose we wanted to check if either of the integers 2 or 3 are present in some lists. This can be achieved with a while loop.

In [64]:
show m:(1 2 3;3 4 5;4 5 6)   / three lists
{
    i:0;
    a:();
    while[i<count x;
            a,:enlist any 2 3 in x[i];
            i+:1];
    a} m


\t:100000 {
            i:0;
            a:();
            while[i<count x;
                  a,:enlist any 2 3 in x[i];
                  i+:1];
            a} m

1 2 3
3 4 5
4 5 6
110b
377


However, iterators allow neater, more efficient code; easier to read and cheaper to maintain.

In [65]:
any each 2 3 in/: m

\t:100000 any each 2 3 in/: m

110b
193


Similarly we can use the Over iterator to deal easily with situations which would be handled by loops in C-like languages.

Suppose you wanted to join several tables.

In [66]:
//Create a list of tables, of random length
show tt:{1!flip(`sym;`$"pr",x;`$"vol",x)!(`a`b`c;3?50.0;3?100)}each string til 2+rand 10

//one table 
tt[0] 

//Join the tables using a while loop
{a:([]sym:`a`b`c);i:0;while[i<count[x];a:a lj x[i];i+:1];a}tt

//Join the tables using Over
0!(lj/)tt
\c 200 100 

sym| pr0      vol0
---| -------------
a  | 36.24292 54  
b  | 18.38882 45  
c  | 19.97371 28  
sym pr0      vol0 pr1      vol1 pr2      vol2 pr3      vol3
-----------------------------------------------------------
a   36.24292 54   24.22538 91   28.19467 65   32.5097  61  
b   18.38882 45   29.35804 34   44.02434 38   25.01427 5   
c   19.97371 28   46.24605 93   35.28828 24   1.031484 21  
sym pr0      vol0 pr1      vol1 pr2      vol2 pr3      vol3
-----------------------------------------------------------
a   36.24292 54   24.22538 91   28.19467 65   32.5097  61  
b   18.38882 45   29.35804 34   44.02434 38   25.01427 5   
c   19.97371 28   46.24605 93   35.28828 24   1.031484 21  
(+(,`sym)!,`a`b`c)!+`pr0`vol0!(36.24292 18.38882 19.97371;54 45 28)
(+(,`sym)!,`a`b`c)!+`pr1`vol1!(24.22538 29.35804 46.24605;91 34 93)
(+(,`sym)!,`a`b`c)!+`pr2`vol2!(28.19467 44.02434 35.28828;65 38 24)
(+(,`sym)!,`a`b`c)!+`pr3`vol3!(32.5097 25.01427 1.031484;61 5 21)


<img src="../images/qbies.png" style="width: 50px;padding-right:5px;padding-top:5px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Write loops only when you can find no solution using iterators.</i></p>

# Combining iterators

We can calculate [Pascal’s Triangle](https://en.wikipedia.org/wiki/Pascal%27s_triangle) using Scan and Each Prior.

We already have a sufficient grasp of the accumulators to see the Triangle immediately as successive results from some use of Scan.

In [50]:
(+) prior 1 3 3 1       / nearly...

1 4 6 4


In [51]:
(+) prior 1 3 3 1,0     / ...there!

1 4 6 4 1


In [59]:
{(+) prior x,0}\ [7;1]

1
1 1
1 2 1
1 3 3 1
1 4 6 4 1
1 5 10 10 5 1
1 6 15 20 15 6 1
1 7 21 35 35 21 7 1


Notice that the last expression gave us eight rows of the Triangle, not seven. The first item of the result was the original argument, followed by the results of seven successive evaluations.

##### Quiz Time!
Try the Extra Practice Exercises to test all your `prior` knowledge!

In [5]:
{(2#x)#1,x#0} [3]

1 0 0
0 1 0
0 0 1
